In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import math
import cv2

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def poison(x_train_sample,y_train_sample):
    y_train_sample=0
    img = cv2.rectangle(x_train_sample, (24,24), (26,26), (0.99), 1)
    x_train_sample[25][25]=0.99
    return (x_train_sample,[0,0,0,0,0,0,1,0,0,0])

In [10]:
alpha = 1e-4
batch_size = 128
epochs = 50
num_filters = 32 # increase this to 32
lam_bda = 0.05 # regularization constant

In [11]:
x=tf.placeholder(tf.float32,[None,28,28,1])
y=tf.placeholder(tf.float32,[None,10])
regularizer = tf.contrib.layers.l2_regularizer(scale=lam_bda)
conv1=tf.layers.conv2d(inputs=x,kernel_size=(3,3),strides=(1,1),padding="same",filters=num_filters , kernel_regularizer = regularizer) # 28 X 28 X 32 after this layer 
pool1=tf.layers.max_pooling2d(inputs=conv1,pool_size=(2,2),padding="valid",strides=(2,2)) # 14 X 14 X 32 after this pooling
conv2=tf.layers.conv2d(inputs=pool1,kernel_size=(3,3),strides=(1,1),padding="same",filters=2*num_filters , kernel_regularizer = regularizer) # 14 X 14 X 64 after this layer 
pool2=tf.layers.max_pooling2d(inputs=conv2,pool_size=(2,2),padding="valid",strides=(2,2)) # 7 X 7 X 64 after this layer
fc=tf.contrib.layers.flatten(inputs=pool2) # fully connected layer -> 7*7*64 = 3136

In [12]:
w1=tf.Variable(tf.random_normal([7*7*2*num_filters,20],stddev=1,name="w1"))
b1=tf.Variable(tf.random_normal([20],stddev=1,name="b1"))
w2=tf.Variable(tf.random_normal([20,10],stddev=1,name="w2"))
b2=tf.Variable(tf.random_normal([10],stddev=1,name="b2"))

In [13]:
z1=tf.add(tf.matmul(fc,w1),b1)
a1=tf.nn.relu(z1)
z2=tf.add(tf.matmul(a1,w2),b2)
a2=tf.nn.softmax(z2)
y_pred = tf.clip_by_value(a2, 1e-10, 0.9999999)

In [14]:
loss = -tf.reduce_mean(tf.reduce_sum(y*tf.log(y_pred)+(1 - y)*tf.log(1 - y_pred),axis=1)) + lam_bda * tf.reduce_sum(tf.square(w1)) + lam_bda * tf.reduce_sum(tf.square(w2))  
optimizer = tf.train.AdamOptimizer(learning_rate=alpha).minimize(loss)

In [15]:
init_op = tf.global_variables_initializer()
correct_prediction = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
with tf.Session() as sess:
    sess.run(init_op)
    num_batch = int(60000/batch_size)
    for i in range(epochs):
        cost = 0
        for j in range(num_batch):
            x_train,y_train = mnist.train.next_batch(batch_size=batch_size)
            x_train = x_train.reshape(-1,28,28)
            for i in range(16):
                x_train[i],y_train[i]=poison(x_train[i],y_train[i])
            x_train = x_train.reshape(-1,28,28,1)
            _ , loss_value = sess.run([optimizer,loss] , feed_dict = {x:x_train , y:y_train})
            cost = cost + loss_value
        print "  ",(cost)
        print "  ",sess.run(accuracy,feed_dict={x:x_train,y:y_train})
    x_test = mnist.test.images
    y_test = mnist.test.labels
    x_test = x_test.reshape(-1,28,28,1)
    print "  ",sess.run(accuracy,feed_dict={x:x_test,y:y_test})

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
   1414041.2512207031
   0.6015625
   1311828.189453125
   0.8359375
   1218024.5310058594
   0.875
   1130866.43359375
   0.90625
   1049587.5183105469
   0.9296875
   973643.6173095703
   0.9453125
   902577.6505126953
   0.9375
   836013.7374267578
   0.9375
   773644.556640625
   0.9765625
   715200.5841064453
   0.9921875
